# To Infinity and Beyond
### *Novel urban routes for intrepid runners.*

*Heidi M Hurst  
GEOGG152 2018  
Network Analysis*

### Introduction

Today’s runner is more tech savvy than ever.  Armed with GPS-enabled Garmin and Apple watches and shoe-embedded sensors, runners can track in real time everything from pace to heart rate to foot strike angle.  A myriad of apps and websites exist to help them analyze this data: MapMyRun, Strava, MyFitnessPal, etc. Many of these tools help runners discover routes through crowd-sourced databases or randomly generating routes of a given length from a user-specified start point.  Given the proliferation of GPS-enabled fitness services, it is no wonder that a number of patents have been filed for route suggestion engines.







|<img src="images/rg1.png" alt="Route Genius - Example 1" style="width: 200px;"/>|<img src="images/rg2.png" alt="Route Genius - Example 2" style="width: 200px;"/>|<img src="images/rg3.png" alt="Route Genius - Example 3" style="width: 200px;"/>
|----|----|----|  

*Figure 1: Three different random routes created by RouteGenius from the same start point (in red) and target distance parameter (~3 miles).*

Despite the data available to these tools - in some cases, a person’s complete running history - route suggestion engines do not take advantage of prior run data to suggest novel routes.  This coursework aims to rectify that deficiency by creating a route engine that uses past GPX tracks to prioritize never before traveled segments of road, encouraging a runner to explore new areas.

This paper addresses relevant network analysis theory and discusses the requirements of such a system.  Next, it details the design and implementation process, including design decisions made and roadblocks encountered.  Finally, it shows a few examples of generated routes and suggests next steps.

In this paper, “road segments” will refer to a single stretch of road, trail, path, way, etc between two intersections.  A road comprises one or more road segments.